# Module 12 Challenge
## Deliverable 2: Scrape and Analyze Mars Weather Data

In [ ]:
# Import relevant libraries
from splinter import Browser
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
browser = Browser('chrome')

### Step 1: Visit the Website

Use automated browsing to visit the [Mars Temperature Data Site](https://static.bc-edx.com/data/web/mars_facts/temperature.html). Inspect the page to identify which elements to scrape.

   > **Hint** To identify which elements to scrape, you might want to inspect the page by using Chrome DevTools to discover whether the table contains usable classes.


In [ ]:
# Visit the website
# https://static.bc-edx.com/data/web/mars_facts/temperature.html
url = "https://static.bc-edx.com/data/web/mars_facts/temperature.html"
browser.visit(url)

### Step 2: Scrape the Table

Create a Beautiful Soup object and use it to scrape the data in the HTML table.

Note that this can also be achieved by using the Pandas `read_html` function. However, use Beautiful Soup here to continue sharpening your web scraping skills.

In [ ]:
# Create a Beautiful Soup Object
html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [ ]:
# Extract all rows of data
table = soup.find("table", class_="table")

table

### Step 3: Store the Data

Assemble the scraped data into a Pandas DataFrame. The columns should have the same headings as the table on the website. Here’s an explanation of the column headings:

* `id`: the identification number of a single transmission from the Curiosity rover
* `terrestrial_date`: the date on Earth
* `sol`: the number of elapsed sols (Martian days) since Curiosity landed on Mars
* `ls`: the solar longitude
* `month`: the Martian month
* `min_temp`: the minimum temperature, in Celsius, of a single Martian day (sol)
* `pressure`: The atmospheric pressure at Curiosity's location

In [ ]:
# Create an empty list
mars_facts = []


#Assign the web scrape parameters
row_heading = table.find_all('th')
row_data = table.find_all('tr', class_= "data-row")

# Loop through the scraped data to create a list of data in rows and a header list
header = [title.text for title in row_heading]

for rows in row_data:
    data = rows.text.split()
    mars_facts.append(data)

In [ ]:
# Create a Pandas DataFrame by using the list of rows and a list of the column names
df = pd.DataFrame(mars_facts, columns = header)
df = df.set_index('id')

In [ ]:
# Confirm DataFrame was created successfully
df.head()

### Step 4: Prepare Data for Analysis

Examine the data types that are currently associated with each column. If necessary, cast (or convert) the data to the appropriate `datetime`, `int`, or `float` data types.

  > **Hint** You can use the Pandas `astype` and `to_datetime` methods to accomplish this task.


In [ ]:
# Examine data type of each column
df.dtypes

In [ ]:
# Change data types for data analysis
df['terrestrial_date'] = pd.to_datetime(df['terrestrial_date'], errors = 'ignore')
df['sol'] = df['sol'].astype(int)
df['ls'] = df['ls'].astype(int)
df['month'] = df['month'].astype(int)
df['min_temp'] = df['min_temp'].astype(int)
df['pressure'] = df['pressure'].astype(float)

In [ ]:
# Confirm type changes were successful by examining data types again
df.dtypes

### Step 5: Analyze the Data

Analyze your dataset by using Pandas functions to answer the following questions:

1. How many months exist on Mars?
2. How many Martian (and not Earth) days worth of data exist in the scraped dataset?
3. What are the coldest and the warmest months on Mars (at the location of Curiosity)? To answer this question:
    * Find the average the minimum daily temperature for all of the months.
    * Plot the results as a bar chart.
4. Which months have the lowest and the highest atmospheric pressure on Mars? To answer this question:
    * Find the average the daily atmospheric pressure of all the months.
    * Plot the results as a bar chart.
5. About how many terrestrial (Earth) days exist in a Martian year? To answer this question:
    * Consider how many days elapse on Earth in the time that Mars circles the Sun once.
    * Visually estimate the result by plotting the daily minimum temperature.


In [ ]:
# 1. How many months are there on Mars?
x = len(pd.unique(df['month']))

print("Number of months on Mars: ", x)


In [ ]:
# 2. How many Martian days' worth of data are there?
x = df['sol'].max()

print(f'There are', x, f'Martian days worth of data')


In [ ]:
# 3. What is the average low temperature by month?

df1 = df.groupby(['month'])
df_avg_low = df1[["min_temp"]].mean()

df_avg_low


In [ ]:
# Plot the average temperature by month

df_avg_low.plot(xticks = [1,2,3,4,5,6,7,8,9,10,11,12])
plt.show()


In [ ]:
# Identify the coldest and hottest months in Curiosity's location


min_ = int(df_avg_low.idxmin())
max_ = int(df_avg_low.idxmax())

print(f'The coldest month in shuttle location was on month ', min_) 
print(f'& the hottest month was on month', max_)


In [ ]:
# 4. Average pressure by Martian month

df1 = df.groupby(['month'])
df_avg_pres = df1[['pressure']].mean()

df_avg_pres


In [ ]:
# Plot the average pressure by month
df_avg_pres.plot(xticks = [1,2,3,4,5,6,7,8,9,10,11,12])
plt.show()

In [ ]:
# 5. How many terrestrial (earth) days are there in a Martian year?
first_ls =df['ls'].iloc[0]

matching_ls=df.loc[df['ls']==first_ls]

date1 = matching_ls['terrestrial_date'].iloc[0]
date2 = matching_ls['terrestrial_date'].iloc[1]
full_martial_year = date2-date1
print(f'In one Martial year, nearly ',full_martial_year,'pass on Earth')

#Research shows that when the sun's longitude (ls) is at the same position, a martial year has passed.  Locating
# the earth date was done by isolating the first ls observation in the data, then locating other data points 
#that shared the same ls point.  Identifying the first two occurence and finding their difference gives us the number of
#earth days that have passed





On average, the third month has the coldest minimum temperature on Mars, and the eighth month is the warmest. But it is always very cold there in human terms!



Atmospheric pressure is, on average, lowest in the sixth month and highest in the ninth.

The distance from peak to peak is roughly 1425-750, or 675 days. A year on Mars appears to be about 675 days from the plot. Internet search confirms that a Mars year is equivalent to 687 earth days.

### Step 6: Save the Data

Export the DataFrame to a CSV file.

In [ ]:
# Write the data to a CSV
df.to_csv('mars_weather.csv', index = True, encoding='utf-8')

In [ ]:
browser.quit()